**THIS FILE CONTAINS THE CODE FOR PREDICTIONS ON THE FINAL TEST SET USING THE CATBOOST REGRESSOR.**

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
pip install catboost

     |████████████████████████████████| 65.8MB 44kB/s 


In [ ]:
file_1=open('/content/drive/MyDrive/Project Energy Consumption/df_te_red_final_modified.txt','rb')
df_tr_red_final=pickle.load(file_1)

**DROPPING THE FEATURES WHICH WERE NOT USED FOR THE TRAINING PROCESS**

In [ ]:
df_tr_red_final.drop(['cloud_coverage','sea_level_pressure','wind_direction','wind_speed',
                      'is_summer_month','is_pub_holiday'],axis=1,inplace=True)

In [ ]:
df_tr_red_final.drop('row_id',axis=1,inplace=True)

In [ ]:
file_2=open('/content/drive/MyDrive/Project Energy Consumption/catboost_model.txt','rb')
catboost_model=pickle.load(file_2)

**HERE WE ARE PREDICTING ON THE TEST SET AND THEN USING EXPM1 TO GET THE ORIGINAL METER READINGS FOR THE FINAL PREDICTIONS**

In [ ]:
y_test=catboost_model.predict(df_tr_red_final)

In [ ]:
y_test=np.expm1(y_test)

In [ ]:
y_test_round=np.round(y_test,4)

In [ ]:
y_test_df=pd.DataFrame(y_test_round)

In [ ]:
y_test_df.rename(columns={0:'meter_reading'},inplace=True)

In [ ]:
y_test_df['row_id']=y_test_df.index

In [ ]:
y_test_df=y_test_df[['row_id','meter_reading']]

In [ ]:
y_test_df['row_id']=y_test_df['row_id'].astype(int)

**ALL THE METER READINGS WHICH ARE PREDICTED TO BE LESS THAN ZERO ARE SET TO ZERO AS METER READINGS CANNOT BE ZERO.**

In [ ]:
for i in tqdm(y_test_df[y_test_df['meter_reading']<0].index):
  y_test_df['meter_reading'][i]=0

  0%|          | 0/176834 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|██████████| 176834/176834 [00:13<00:00, 12910.18it/s]


**CONVERTING THE TEST FILE INTO CSV FORMAT FOR SUBMISSION**

In [ ]:
y_test_df.to_csv('catboost.csv',index=False,header=True)

**FINAL PREDICTED METER READINGS BY THE RF MODEL**

In [ ]:
y_test_df.head(10)

,row_id,meter_reading
0,0,149.8095
1,1,50.1822
2,2,20.7931
3,3,252.3720
4,4,703.6323
5,5,42.0583
6,6,102.4726
7,7,768.0829
8,8,271.4789
9,9,491.9055
